In [2]:
#q1
with zopen("/etc/os-release", "r") as f:
    os_release = f.read()
os_release

'PRETTY_NAME="Ubuntu 24.04.2 LTS"\nNAME="Ubuntu"\nVERSION_ID="24.04"\nVERSION="24.04.2 LTS (Noble Numbat)"\nVERSION_CODENAME=noble\nID=ubuntu\nID_LIKE=debian\nHOME_URL="https://www.ubuntu.com/"\nSUPPORT_URL="https://help.ubuntu.com/"\nBUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"\nPRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"\nUBUNTU_CODENAME=noble\nLOGO=ubuntu-logo\n'

In [122]:
#q2
import subprocess
lscpu = subprocess.check_output(["lscpu"]).decode("utf-8")
lscpu

'Architecture:                         x86_64\nCPU op-mode(s):                       32-bit, 64-bit\nAddress sizes:                        46 bits physical, 48 bits virtual\nByte Order:                           Little Endian\nCPU(s):                               2\nOn-line CPU(s) list:                  0,1\nVendor ID:                            GenuineIntel\nModel name:                           Intel(R) Xeon(R) CPU @ 2.20GHz\nCPU family:                           6\nModel:                                79\nThread(s) per core:                   2\nCore(s) per socket:                   1\nSocket(s):                            1\nStepping:                             0\nBogoMIPS:                             4399.99\nFlags:                                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pci

In [123]:
import pyarrow
from pyarrow import fs 
import pyarrow.parquet as pq 

gcs = fs.GcsFileSystem() 
table = pq.read_table("./../wi-schools-raw.parquet")

bucket = "cs544_p8_uday"
bucket_path = "cs544_p8_uday/wi-schools-raw.parquet"

with gcs.open_output_stream(bucket_path) as out_stream:
    pq.write_table(table, out_stream)
#table.to_pandas().head()

In [21]:
#q3

all_paths = gcs.get_file_info(fs.FileSelector(bucket, recursive = False))
[path.base_name for path in all_paths]


['wi-schools.parquet']

In [22]:
#q4
info = gcs.get_file_info(f"{bucket}/wi-schools-raw.parquet")
info.mtime_ns

1746146544062000000

In [ ]:
from google.cloud import bigquery 
%load_ext bigquery_magics 
#manually created the private p8 dataset
from google.cloud import dataform
client = dataform.DataformClient()

sqlx_wi_counties = """
config {
  type: "table",
  name: "wi_counties",
}

SELECT county_name, county_fips_code, county_geom
FROM bigquery-public-data.geo_us_boundaries.counties
"""
with open("definitions/wi_counties.sqlx", "rb") as f:
    client.write_file(dataform.WriteFileRequest(
        workspace="projects/<PROJECT_ID>/locations/us-central1/repositories/wi_school_repo/workspaces/main",
        path="definitions/wi_counties.sqlx",
        contents=bytes(sqlx_wi_counties, "utf-8")
    )) 

sqlx_schools = """
config {
  type: "operations",
  name: "schools",
  hasOutput: true 
}

LOAD DATA OVERWRITE `cs544-458521`.p8.schools
FROM FILES (
  uris = ["gs://cs544_p8_uday/wi-schools-raw.parquet"],
  format = 'parquet'
  )
"""

with open("definitions/schools.sqlx", "rb") as f:
    client.write_file(dataform.WriteFileRequest(
        workspace="projects/cs544-458521/locations/us-central1/repositories/cs544_p8_dataform_repo/workspaces/p8_repoID_1",
        path="definitions/schools.sqlx",
        contents=bytes(sqlx_schools, "utf-8")
    )) 

sqlx_wi_county_schools = """
config {
  type: "table",
  name: "wisconsin_county_schools",
}

SELECT 
  s.*, 
  ST_GEOGPOINT(s.longitude, s.latitude) AS location, 
  c.county_name
  FROM ${ref("schools")} AS s
  INNER JOIN ${ref("wi_counties")} AS c
  ON ST_WITHIN(ST_GEOGPOINT(s.longitude, s.latitude), c.county_geom)
"""
with open("definitions/wi_county_schools.sqlx", "rb") as f:
    client.write_file(dataform.WriteFileRequest(
        workspace="projects/cs544-458521/locations/us-central1/repositories/cs544_p8_dataform_repo/workspaces/p8_repoID_1",
        path="definitions/wi_county_schools.sqlx",
        contents=bytes(sqlx_wi_county_schools, "utf-8")
    )) 

compilation_response = client.create_compilation_result(
    request={
        "parent": "projects/cs544-458521/locations/us-central1/repositories/cs544_p8_dataform_repo",
        "compilation_result": {
            "git_commitish": "HEAD",  
            "workspace": "projects/cs544-458521/locations/us-central1/repositories/cs544_p8_dataform_repo/workspaces/p8_repoID_1"
        }
    }
)

compilation_result_name = compilation_response.name
compilation_result_name 

The bigquery_magics extension is already loaded. To reload it, use:
  %reload_ext bigquery_magics


'projects/cs544-458521/locations/us-central1/repositories/cs544_p8_dataform_repo/compilationResults/5f7e6ccc-57bc-4968-b148-865656d00adc'

In [69]:
%%bigquery
#q5
SELECT COUNT(*)
FROM `cs544-458521.p8.wisconsin_county_schools`


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,2961


In [70]:
%%bigquery
#q6
SELECT COUNT(*)
FROM `cs544-458521.p8.wisconsin_county_schools`
WHERE agency_type = "Public school"

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,2116


In [88]:
#q7
my_dict = {
  "schools": [],
  "wi_counties": [],
  "wisconsin_county_schools": ["schools", "wi_counties"]
}
my_dict

{'schools': [],
 'wi_counties': [],
 'wisconsin_county_schools': ['schools', 'wi_counties']}

In [89]:
#q8
query = """
SELECT county_name, COUNT(*) AS num_schools
FROM `cs544-458521.p8.wisconsin_county_schools`
GROUP BY county_name
"""

df = bq.query(query).to_dataframe()
schools_per_county = dict(zip(df["county_name"], df["num_schools"].astype(int)))
schools_per_county

{'Brown': 123,
 'Shawano': 29,
 'Price': 9,
 'Waukesha': 177,
 'Kenosha': 56,
 'Milwaukee': 420,
 'Racine': 68,
 'Rock': 72,
 'Dane': 217,
 'Portage': 38,
 'Fond du Lac': 55,
 'Ozaukee': 38,
 'Calumet': 30,
 'Washington': 56,
 'Dodge': 51,
 'Jefferson': 52,
 'Manitowoc': 44,
 'Sheboygan': 63,
 'Walworth': 56,
 'Green': 23,
 'Lafayette': 18,
 'Iowa': 16,
 'Sauk': 42,
 'Richland': 14,
 'Grant': 39,
 'Crawford': 17,
 'Columbia': 41,
 'Green Lake': 15,
 'Juneau': 26,
 'Marquette': 10,
 'Pierce': 29,
 'St. Croix': 35,
 'Polk': 28,
 'Marinette': 29,
 'Outagamie': 96,
 'Oconto': 20,
 'Kewaunee': 15,
 'Door': 18,
 'Marathon': 72,
 'Langlade': 13,
 'Wood': 47,
 'Clark': 38,
 'Taylor': 13,
 'Lincoln': 16,
 'Oneida': 17,
 'Vilas': 13,
 'La Crosse': 59,
 'Trempealeau': 27,
 'Vernon': 29,
 'Monroe': 28,
 'Buffalo': 11,
 'Eau Claire': 54,
 'Chippewa': 35,
 'Dunn': 22,
 'Pepin': 7,
 'Jackson': 10,
 'Barron': 40,
 'Washburn': 16,
 'Ashland': 12,
 'Sawyer': 13,
 'Rusk': 11,
 'Douglas': 18,
 'Winnebago'

In [117]:
#q9
config = bigquery.QueryJobConfig(use_query_cache=False)
q = bq.query("""
SELECT county_name, COUNT(*) AS num_schools
FROM `cs544-458521.p8.wisconsin_county_schools`
GROUP BY county_name
""", job_config = config)


q.total_bytes_billed 
TB = 1024**4
runs = TB // q.total_bytes_billed
runs

104857

In [115]:
%%bigquery q
SELECT
  m.school_name AS middle_school,
  MIN_BY(h.school_name, ST_DISTANCE(
    ST_GEOGPOINT(m.longitude, m.latitude),
    ST_GEOGPOINT(h.longitude, h.latitude)
  )) AS closest_high_school
FROM `cs544-458521.p8.wisconsin_county_schools` m
JOIN `cs544-458521.p8.wisconsin_county_schools` h
ON m.county_name = h.county_name
WHERE m.county_name = "Dane"
  AND m.agency_type = "Public school"
  AND h.agency_type = "Public school"
  AND m.lowest_grade <= "06" AND m.highest_grade >= "06"
  AND h.lowest_grade <= "09" AND h.highest_grade >= "12"
GROUP BY m.school_name


Query is running:   0%|          |

Downloading:   0%|          |

In [116]:
#q10
q10_dict = dict(zip(q["middle_school"], q["closest_high_school"]))
q10_dict

{'Nikolay Middle': 'Koshkonong Trails School',
 'Glacial Drumlin School': 'LaFollette High',
 'Glacier Creek Middle': 'Middleton High',
 'Harvest Intermediate School': 'De Forest High',
 'Indian Mound Middle': 'McFarland High',
 'Marshall Elementary': 'Marshall High',
 'Wisconsin Heights Middle': 'Wisconsin Heights High',
 'Kromrey Middle': 'Middleton High',
 'Mount Horeb Middle': 'Mount Horeb High',
 'Rome Corners Intermediate': 'Oregon High',
 'River Bluff Middle': 'Stoughton High',
 'Central Heights Middle': 'Prairie Phoenix Academy',
 'Patrick Marsh Middle': 'Prairie Phoenix Academy',
 'Prairie View Middle': 'Sun Prairie West High',
 'Badger Ridge Middle': 'Verona Area High',
 'Waunakee Intermediate': 'Waunakee High',
 'Metro School': 'Metro School',
 'Innovative and Alternative Middle': 'Innovative High',
 "O'Keeffe Middle": 'Innovative High',
 'Black Hawk Middle': 'Shabazz-City High',
 'Sherman Middle': 'Shabazz-City High',
 'Spring Harbor Middle': 'Capital High',
 'Hamilton Midd